# New Section

In [57]:
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')

In [56]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Import libraries**

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



**Load the data**

In [ ]:
sales = pd.read_csv("/content/drive/My Drive/M5 Forcasting/sales_train_evaluation.csv")
price = pd.read_csv("/content/drive/My Drive/M5 Forcasting/sell_prices.csv")
cal = pd.read_csv("/content/drive/My Drive/M5 Forcasting/calendar.csv")

In [ ]:
price["id"] = price["item_id"] + "_" + price["store_id"] + "_evaluation"
cal["d"] = cal["d"].apply(lambda a: int(a.split("_")[1]))

**Calculate the weights for level 12 series**

In [ ]:
#https://www.kaggle.com/qcw171717/other-naive-forecasts-submission-score/notebook

for day in tqdm(range(1886, 1914)):  
    wk_id = list(cal[cal["d"]==day]["wm_yr_wk"])[0]
    wk_price = price[price["wm_yr_wk"]==wk_id]
    df_sales = sales.merge(wk_price[["sell_price", "id"]], on=["id"], how='inner')
    df_sales["unit_sales_" + str(day)] = df_sales["sell_price"] * df_sales["d_" + str(day)]
    df_sales.drop(columns=["sell_price"], inplace=True)

100%|██████████| 28/28 [00:09<00:00,  3.04it/s]


In [ ]:
col = [a for a in df_sales.columns if a.find("unit_sales")==0]
df_sales["sales"] = df_sales[col]
df_sales["weight"] = df_sales["sales"] / df_sales["sales"].sum()
df_sales.drop(columns=["sales", col[0]], axis=1, inplace=True)
df_sales["weight"] /= 12

In [ ]:
df_sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,weight
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,3,0,1,1,1,3,0,1,1,0,0,0,2,0,3,5,0,0,1,1,0,2,1,2,2,1,0,2,4,0,0,0,0,3,3,0,1,0.000004
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,2,1,1,0,0,0,0,0,0.000000
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,2,1,2,1,1,1,0,1,1,1,0,0,1,1,0,2,1,0,0,0,0,2,1,3,0,0,1,0,1,0,2,0,0,0,2,3,0,1,0.000002
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,5,4,1,0,1,3,7,2,0,0,1,2,4,1,6,4,0,0,0,2,2,4,2,1,1,1,1,1,0,4,0,1,3,0,2,6,0.000005
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,2,1,1,0,1,1,2,2,2,4,1,0,2,3,1,0,3,2,3,1,1,3,2,3,2,2,2,2,0,0,0,2,1,0,0,2,1,0,0.000006


In [ ]:
day_coloumns = [a for a in df_sales.columns if 'd_' in a]
forecast_days = 28

**Train/Val split**

In [ ]:
train_dataset = df_sales[day_coloumns[:1913]]
val_dataset = df_sales[day_coloumns[1913:]]

**Simple Moving Averages Model**

In [ ]:
def sim_mov_avgs(train_dataset, forecast_days, wind_size):

    preds = []
    for i in range(forecast_days):
        if i == 0:
            preds.append(np.mean(train_dataset[train_dataset.columns[-wind_size:]].values, axis=1))
        if i < forecast_days and i > 0:
            preds.append(0.5 * (np.mean(train_dataset[train_dataset.columns[-wind_size + i:]].values, axis=1) + \
                                  np.mean(preds[:i], axis=0)))
            
    return preds   

In [ ]:
predictions = sim_mov_avgs(train_dataset, forecast_days, 28)
for d, i in enumerate(range(1914, 1942)):
    df_sales['F_' + str(i)] = predictions[d]

In [ ]:
df_sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,weight,F_1914,F_1915,F_1916,F_1917,F_1918,F_1919,F_1920,F_1921,F_1922,F_1923,F_1924,F_1925,F_1926,F_1927,F_1928,F_1929,F_1930,F_1931,F_1932,F_1933,F_1934,F_1935,F_1936,F_1937,F_1938,F_1939,F_1940,F_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,2,4,0,0,0,0,3,3,0,1,0.000004,0.964286,0.963624,0.981978,1.004981,1.031025,1.059807,1.091384,1.102173,1.137454,1.071338,1.048180,0.991326,1.018648,1.017931,0.981576,1.015524,1.056705,1.106731,1.117908,1.131179,1.022344,1.093240,1.106273,1.124252,1.150831,0.861147,1.024074,1.023628
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,2,1,1,0,0,0,0,0,0.000000,0.071429,0.054233,0.050646,0.049385,0.049045,0.049213,0.049723,0.050500,0.051511,0.052743,0.054199,0.055895,0.057855,0.060117,0.062732,0.065769,0.069323,0.073523,0.078551,0.084671,0.092277,0.101984,0.114818,0.132612,0.034016,0.033335,0.032694,0.032089
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,2,0,0,0,2,3,0,1,0.000002,0.571429,0.582011,0.596052,0.611582,0.628467,0.646780,0.666663,0.664499,0.685468,0.708790,0.734754,0.734333,0.763784,0.797229,0.835423,0.879370,0.930416,0.944940,0.910611,0.865595,0.868955,0.800646,0.790026,0.774300,0.749836,0.874839,0.877246,0.879519
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,4,0,1,3,0,2,6,0.000005,1.821429,1.855159,1.899916,1.949417,2.003240,1.996394,2.028645,2.039586,2.074612,2.060525,2.097557,2.168583,2.187294,2.107014,2.046049,2.049642,2.137033,2.106265,2.217454,2.300036,2.466146,2.324095,2.210669,2.346669,2.676946,3.063408,3.331419,2.103059
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,2,1,0,0,2,1,0,0.000006,1.357143,1.363757,1.391763,1.345444,1.286430,1.304888,1.307149,1.239755,1.262271,1.264074,1.295023,1.302409,1.342504,1.356254,1.372102,1.352007,1.410718,1.436071,1.466382,1.447676,1.485096,1.533069,1.680045,1.787001,1.945605,2.040026,2.218744,2.733211


For Simple Moving Averages model, manual hyperparameter tuning is done for hyperparameter i.e. window size and window size of length 28 is found.

**Make aggregation dataframe**

In [ ]:
aggregation_level = {2: ["state_id"], 3: ["store_id"], 4: ["cat_id"], 5: ["dept_id"], 
              6: ["state_id", "cat_id"], 7: ["state_id", "dept_id"], 8: ["store_id", "cat_id"], 9: ["store_id", "dept_id"],
              10: ["item_id"], 11: ["item_id", "state_id"]}

In [ ]:
data = df_sales[[a for a in df_sales.columns if a.find("d_") == 0 or a.find("F_") == 0]]
data = data.sum()

aggregated_df = pd.DataFrame(data).transpose()    
aggregated_df["level"] = 1
aggregated_df["weight"] = 1/12    
columns = aggregated_df.columns  

for lev in aggregation_level:
    df_t = df_sales.groupby(by=aggregation_level[lev]).sum().reset_index()
    df_t["level"] = lev
    aggregated_df = aggregated_df.append(df_t[columns])
                

In [ ]:
aggregated_df

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,F_1914,F_1915,F_1916,F_1917,F_1918,F_1919,F_1920,F_1921,F_1922,F_1923,F_1924,F_1925,F_1926,F_1927,F_1928,F_1929,F_1930,F_1931,F_1932,F_1933,F_1934,F_1935,F_1936,F_1937,F_1938,F_1939,F_1940,F_1941,level,weight
0,32631.0,31749.0,23783.0,25412.0,19146.0,29211.0,28010.0,37932.0,32736.0,25572.0,23071.0,23658.0,23756.0,26445.0,34833.0,36380.0,21804.0,24070.0,21443.0,20318.0,23721.0,31689.0,29283.0,23966.0,20501.0,20757.0,20277.0,22529.0,29908.0,28707.0,21240.0,22872.0,22046.0,23475.0,23572.0,31202.0,34876.0,24562.0,22752.0,22560.0,...,44073.0,54308.0,59921.0,42362.0,38777.0,37096.0,36963.0,42552.0,51518.0,54338.0,42272.357143,42387.752646,42582.873601,42834.543898,43065.690911,43078.495733,42908.173131,42605.825266,42649.407021,42721.939379,42822.575159,42969.379080,42979.667207,42551.415904,42132.789146,42077.519251,42157.179306,42435.712357,42665.536004,42717.627162,42372.848983,41686.340673,41889.749598,42238.160838,43055.115821,44489.506838,46003.638116,46274.385559,1,0.083333
0,14195.0,13805.0,10108.0,11047.0,9925.0,11322.0,12251.0,16610.0,14696.0,11822.0,10933.0,10350.0,11204.0,11354.0,14610.0,15630.0,9044.0,9496.0,8844.0,7835.0,10025.0,13870.0,13465.0,11997.0,9130.0,8941.0,8514.0,9847.0,13652.0,12491.0,9196.0,9782.0,9537.0,9710.0,10444.0,13784.0,16057.0,10953.0,10299.0,9951.0,...,18471.0,23037.0,24704.0,17721.0,16150.0,15678.0,16297.0,17430.0,23103.0,24644.0,18006.107143,18052.849868,18118.008483,18209.854249,18286.206635,18295.454812,18224.434038,18079.184421,18068.356228,18104.943455,18165.353300,18246.510146,18270.448449,18110.735047,17894.373081,17890.863209,17984.990080,18127.624062,18281.291575,18407.176300,18296.869115,17973.467469,18031.676542,18243.173428,18645.707315,19432.979836,20360.031506,20738.104996,2,0.040149
1,9438.0,9630.0,6778.0,7381.0,5912.0,9006.0,6226.0,9440.0,9376.0,7319.0,6224.0,6740.0,6109.0,7202.0,10081.0,11725.0,6719.0,7460.0,6606.0,6412.0,6711.0,9150.0,9789.0,7297.0,6628.0,5814.0,6175.0,6286.0,8586.0,9086.0,6555.0,7082.0,6145.0,6639.0,6584.0,9238.0,10423.0,7623.0,6335.0,6941.0,...,12320.0,12366.0,16249.0,12228.0,11370.0,10375.0,9162.0,12303.0,13681.0,14815.0,11046.571429,11060.341270,11116.708944,11174.290274,11227.759823,11229.806304,11198.517110,11135.928225,11150.620211,11168.790433,11193.411146,11214.044566,11232.366239,11166.659845,11072.064851,11061.442176,11081.124672,11138.788563,11180.345447,11204.100099,11130.342041,10944.024373,11011.675334,11004.754856,11161.134963,11426.612264,11708.773566,11723.240723,2,0.021284
2,8998.0,8314.0,6897.0,6984.0,3309.0,8883.0,9533.0,11882.0,8664.0,6431.0,5914.0,6568.0,6443.0,7889.0,10142.0,9025.0,6041.0,7114.0,5993.0,6071.0,6985.0,8669.0,6029.0,4672.0,4743.0,6002.0,5588.0,6396.0,7670.0,7130.0,5489.0,6008.0,6364.0,7126.0,6544.0,8180.0,8396.0,5986.0,6118.0,5668.0,...,13282.0,18905.0,18968.0,12413.0,11257.0,11043.0,11504.0,12819.0,14734.0,14879.0,13219.678571,13274.561508,13348.156174,13450.399376,13551.724454,13553.234617,13485.221982,13390.712620,13430.430581,13448.205491,13463.810713,13508.824368,13476.852519,13274.021012,13166.351214,13125.213866,13091.064554,13169.299732,13203.898982,13106.350763,12945.637827,12768.848832,12846.397722,12990.232554,13248.273542,13629.914738,13934.833044,13813.039840,2,0.021901
0,4337.0,4155.0,2816.0,3051.0,2630.0,3276.0,3450.0,5437.0,4340.0,3157.0,2995.0,2710.0,2928.0,3078.0,4316.0,4354.0,2757.0,2430.0,2272.0,2009.0,3015.0,4553.0,3966.0,3264.0,2441.0,2377.0,2414.0,2701.0,3719.0,3548.0,2246.0,2549.0,2559.0,2568.0,2978.0,4096.0,4832.0,3138.0,2997.0,2772.0,...,4951.0,6245.0,6707.0,4568.0,3949.0,3995.0,4136.0,4433.0,5764.0,6289.0,4569.821429,4582.132937,4600.719361,4630.572288,4648.884918,4648.082658,4624.570829,4581.079839,4572.341516,4582.602075,4597.929281,462

In [ ]:
print(df_sales.shape[0], aggregated_df.shape[0], df_sales.shape[0] + aggregated_df.shape[0])

30490 12350 42840


In [ ]:
aggregated_df["weight"].sum() + df_sales["weight"].sum()

1.0000000000000004

**Calculate RMSSE and WRMSSE**

In [ ]:
#function to calculate rmsse 

h = 28
n = 1913

def RMSSE(ground_truth, forecast, train_series):
    
    num = ((ground_truth - forecast)**2).sum(axis=1)
    den = 1/(n-1) * ((train_series[:, 1:] - train_series[:, :-1]) ** 2).sum(axis=1)
    rmsse = (1/h * num/den) ** 0.5

    return rmsse

In [ ]:
train_columns = [a for a in df_sales.columns if a.find("d_") == 0 and int(a.split('_')[1]) < 1914]
actual_value_columns = [a for a in df_sales.columns if a.find("d_") == 0 and int(a.split('_')[1]) in range(1914, 1942)]
forecast_value_columns = [a for a in df_sales.columns if a.find("F_") == 0]

In [ ]:
ground_truth_df = np.array(df_sales[actual_value_columns])
forecast_df = np.array(df_sales[forecast_value_columns])
train_series_df = np.array(df_sales[train_columns])

ground_truth_agg_df = np.array(aggregated_df[actual_value_columns])
forecast_agg_df = np.array(aggregated_df[forecast_value_columns])
train_series_agg_df = np.array(aggregated_df[train_columns])

            
df_sales["rmsse"] = RMSSE(ground_truth_df, forecast_df, train_series_df)
aggregated_df["rmsse"] = RMSSE(ground_truth_agg_df, forecast_agg_df, train_series_agg_df)

df_sales["wrmsse"] = df_sales["weight"] * df_sales["rmsse"]
aggregated_df["wrmsse"] = aggregated_df["weight"] * aggregated_df["rmsse"]


In [ ]:
WRMSSE = df_sales["wrmsse"].sum() + aggregated_df["wrmsse"].sum()
print("The WRMSSE for all 42840 series is", WRMSSE)

The WRMSSE for all 42840 series is 1.0666065702168306


**Make submission file**

In [ ]:
submit_df1 = df_sales[["id"]]

for i in range(0, 28):
    submit_df1["F" + str(i+1)] = predictions[i]

submit_df1["id"] = submit_df1["id"].apply(lambda x: x.replace('evaluation', 'validation'))    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
train_dataset = df_sales[day_coloumns[:1941]]
predictions = sim_mov_avgs(train_dataset, forecast_days, 30)

In [ ]:
submit_df2 = df_sales[["id"]]
for i in range(0, 28):
    submit_df2["F" + str(i+1)] = predictions[i]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
submit_df = submit_df1.append(submit_df2).reset_index(drop=True)

In [ ]:
submit_df

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.964286,0.963624,0.981978,1.004981,1.031025,1.059807,1.091384,1.102173,1.137454,1.071338,1.048180,0.991326,1.018648,1.017931,0.981576,1.015524,1.056705,1.106731,1.117908,1.131179,1.022344,1.093240,1.106273,1.124252,1.150831,0.861147,1.024074,1.023628
1,HOBBIES_1_002_CA_1_validation,0.071429,0.054233,0.050646,0.049385,0.049045,0.049213,0.049723,0.050500,0.051511,0.052743,0.054199,0.055895,0.057855,0.060117,0.062732,0.065769,0.069323,0.073523,0.078551,0.084671,0.092277,0.101984,0.114818,0.132612,0.034016,0.033335,0.032694,0.032089
2,HOBBIES_1_003_CA_1_validation,0.571429,0.582011,0.596052,0.611582,0.628467,0.646780,0.666663,0.664499,0.685468,0.708790,0.734754,0.734333,0.763784,0.797229,0.835423,0.879370,0.930416,0.944940,0.910611,0.865595,0.868955,0.800646,0.790026,0.774300,0.749836,0.874839,0.877246,0.879519
3,HOBBIES_1_004_CA_1_validation,1.821429,1.855159,1.899916,1.949417,2.003240,1.996394,2.028645,2.039586,2.074612,2.060525,2.097557,2.168583,2.187294,2.107014,2.046049,2.049642,2.137033,2.106265,2.217454,2.300036,2.466146,2.324095,2.210669,2.346669,2.676946,3.063408,3.331419,2.103059
4,HOBBIES_1_005_CA_1_validation,1.357143,1.363757,1.391763,1.345444,1.286430,1.304888,1.307149,1.239755,1.262271,1.264074,1.295023,1.302409,1.342504,1.356254,1.372102,1.352007,1.410718,1.436071,1.466382,1.447676,1.485096,1.533069,1.680045,1.787001,1.945605,2.040026,2.218744,2.733211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.633333,0.644253,0.640825,0.653069,0.667589,0.683907,0.660248,0.631721,0.644116,0.658837,0.675895,0.642780,0.659857,0.679727,0.671470,0.661588,0.649841,0.674361,0.703706,0.693561,0.730767,0.721304,0.771199,0.621670,0.668242,0.634877,0.584207,0.665981
60976,FOODS_3_824_WI_3_evaluation,0.300000,0.287931,0.289840,0.294443,0.281142,0.265336,0.247391,0.249130,0.252087,0.256120,0.261171,0.240926,0.245508,0.192325,0.193334,0.195223,0.198051,0.201922,0.206997,0.213511,0.221810,0.232402,0.246059,0.264002,0.288264,0.322498,0.373989,0.292989
60977,FOODS_3_825_WI_3_evaluation,0.766667,0.762644,0.775185,0.791490,0.810075,0.810606,0.809722,0.829525,0.806335,0.802665,0.798246,0.819378,0.843717,0.842186,0.871194,0.904654,0.907634,0.948812,0.872243,0.778715,0.763792,0.801162,0.849242,0.840873,0.745974,0.811055,0.785842,0.910178
60978,FOODS_3_826_WI_3_evaluation,1.366667,1.372989,1.345628,1.347547,1.313719,1.334655,1.340100,1.325124,1.330743,1.361430,1.346930,1.356280,1.367020,1.349887,1.392276,1.375033,1.390099,1.407714,1.470107,1.454147,1.483702,1.519805,1.564809,1.622406,1.448725,0.999751,1.067063,1.019525


In [ ]:
submit_df.to_csv("submission.csv", index=False)

In [ ]:
print("Leaderboard Public score for SMA model is 1.02947")
print("Leaderboard Private score for SMA model is 0.93062")

Leaderboard Public score for SMA model is 1.02947
Leaderboard Private score for SMA model is 0.93062


In [58]:
%%capture
!wget -nc https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
from colab_pdf import colab_pdf
colab_pdf('simple_model.ipynb')